In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Load the CSV file
df = pd.read_csv("Periodic_Table_of_Elements.csv")
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')

# Coordinates for main table
df['x'] = df['Group'] - 1
df['y'] = 9 - df['Period']

# Shift Lanthanides and Actinides to new positions
lanth_mask = df['Type'] == 'Lanthanide'
actin_mask = df['Type'] == 'Actinide'

# x positions
df.loc[lanth_mask, 'x'] = df[lanth_mask].reset_index().index + 2
df.loc[lanth_mask, 'y'] = 0  # Lanthanides row

df.loc[actin_mask, 'x'] = df[actin_mask].reset_index().index + 2
df.loc[actin_mask, 'y'] = -1  # Actinides row

# Define color map
type_colors = {
    'Alkali Metal':           '#ff6b6b',
    'Alkaline Earth Metal':   '#ffb347',
    'Transition Metal':       '#d5dbdb',
    'Lanthanide':             '#6fb3f2',
    'Actinide':               '#d2a8e3',
    'Metalloid':              '#2ecc71',
    'Nonmetal':               '#66ff99',
    'Halogen':                '#5dade2',
    'Noble Gas':              '#d291ff',
    'Transactinide':          '#d5dbdb',
    'Metal':                  '#cfd8dc'
}
default_color = '#ffffff'

# Year range
min_year = int(df['Year'].min())
max_year = int(df['Year'].max())

# Animation frames
frames = []
for year in range(min_year, max_year + 1):
    frame_shapes = []
    frame_texts = []

    for _, row in df.iterrows():
        if pd.notna(row['x']) and pd.notna(row['y']):
            color = type_colors.get(row['Type'], default_color) if row['Year'] <= year else '#ffffff'

            # Rectangle for element
            frame_shapes.append(dict(
                type='rect',
                x0=row['x'], x1=row['x'] + 1,
                y0=row['y'], y1=row['y'] + 1,
                line=dict(color='gray', width=1),
                fillcolor=color,
                layer='below'
            ))

            # Text on tile
            frame_texts.append(
                go.Scatter(
                    x=[row['x'] + 0.5], y=[row['y'] + 0.5],
                    text=f"<b>{row['Symbol']}</b><br>{row['AtomicNumber']}",
                    mode="text",
                    textposition="middle center",
                    hovertext=f"{row['Element']} ({row['AtomicNumber']})",
                    hoverinfo='text',
                    showlegend=False
                )
            )

    frames.append(go.Frame(data=frame_texts, layout=go.Layout(shapes=frame_shapes), name=str(year)))

# Initial display (first year)
initial_shapes = frames[0].layout.shapes
initial_data = frames[0].data

# Layout
layout = go.Layout(
    title=dict(
        text="Periodic Table by Discovery Year",
        x=0.5,            # Center horizontally (0 = left, 1 = right)
        y=0.85,           # Slightly below top (1 = very top)
        xanchor='center',
        yanchor='top',
        font=dict(size=24, family='Arial', color='black'),
        pad=dict(t=10)    # Top padding (optional, fine tuning)
    ),
    paper_bgcolor='#f0ffff',   # <-- Full canvas background
    plot_bgcolor='#f0ffff',    # <-- Plot (axes) background
    xaxis=dict(visible=False, range=[-0.5, 18.5]),
    yaxis=dict(visible=False, range=[-3.5, 10]),
    height=750,
    width=1100,
    shapes=initial_shapes,
    sliders=[{
        "steps": [{
            "method": "animate",
            "label": str(year),
            "args": [[str(year)], {"mode": "immediate", "frame": {"duration": 300, "redraw": True}, "transition": {"duration": 0}}],
        } for year in range(min_year, max_year + 1)],
        "transition": {"duration": 0},
        "x": 0.1,
        "xanchor": "left",
        "y": 0.2,
        "yanchor": "top",
        "len": 0.9,       # length of the slider (0.0 to 1.0)
    }],
    updatemenus=[{
        "type": "buttons",
        "showactive": False,
        "y": 0.15,
        "x": 0,
        "xanchor": "left",
        "buttons": [
            {"label": "Play", "method": "animate", "args": [None, {"frame": {"duration": 500, "redraw": True}, "fromcurrent": True}]},
            {"label": "Pause", "method": "animate", "args": [[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}]}
        ]
    }]
)

# Create and show figure
fig = go.Figure(data=initial_data, layout=layout, frames=frames)
fig.show()